In [1]:
# import dependencies
import pandas as pd
import nba_api
import time
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, teamgamelogs, playercareerstats, playergamelogs, \
leaguegamelog, leaguegamefinder, playerdashboardbyyearoveryear
from nba_api.stats.library.parameters import SeasonAll
from itertools import zip_longest
import os
import numpy as np

In [2]:
pwd = os.getcwd()

In [3]:
custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [4]:
# Get all players dictionary.
all_players_dict = players.get_players()
all_players_dict

[{'id': 76001,
  'full_name': 'Alaa Abdelnaby',
  'first_name': 'Alaa',
  'last_name': 'Abdelnaby',
  'is_active': False},
 {'id': 76002,
  'full_name': 'Zaid Abdul-Aziz',
  'first_name': 'Zaid',
  'last_name': 'Abdul-Aziz',
  'is_active': False},
 {'id': 76003,
  'full_name': 'Kareem Abdul-Jabbar',
  'first_name': 'Kareem',
  'last_name': 'Abdul-Jabbar',
  'is_active': False},
 {'id': 51,
  'full_name': 'Mahmoud Abdul-Rauf',
  'first_name': 'Mahmoud',
  'last_name': 'Abdul-Rauf',
  'is_active': False},
 {'id': 1505,
  'full_name': 'Tariq Abdul-Wahad',
  'first_name': 'Tariq',
  'last_name': 'Abdul-Wahad',
  'is_active': False},
 {'id': 949,
  'full_name': 'Shareef Abdur-Rahim',
  'first_name': 'Shareef',
  'last_name': 'Abdur-Rahim',
  'is_active': False},
 {'id': 76005,
  'full_name': 'Tom Abernethy',
  'first_name': 'Tom',
  'last_name': 'Abernethy',
  'is_active': False},
 {'id': 76006,
  'full_name': 'Forest Able',
  'first_name': 'Forest',
  'last_name': 'Able',
  'is_active': Fa

In [5]:
# flatten data aka melting
all_players_df = pd.json_normalize(all_players_dict)
all_players_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4496,1627790,Ante Zizic,Ante,Zizic,True
4497,78647,Jim Zoet,Jim,Zoet,False
4498,78648,Bill Zopf,Bill,Zopf,False
4499,1627826,Ivica Zubac,Ivica,Zubac,True


In [6]:
salaries_path = os.path.join(pwd, 'Resources', 'salaries_1985to2022.csv')
salaries_df = pd.read_csv(salaries_path)
salaries_df 

,season,season_end,season_start,name,team,position,salary
0,1990-91,1991,1990,Alaa Abdelnaby,Portland Trail Blazers,Power Forward,395000
1,1991-92,1992,1991,Alaa Abdelnaby,Portland Trail Blazers,Power Forward,494000
2,1992-93,1993,1992,Alaa Abdelnaby,Boston Celtics,Power Forward,500000
3,1993-94,1994,1993,Alaa Abdelnaby,Boston Celtics,Power Forward,805000
4,1994-95,1995,1994,Alaa Abdelnaby,Sacramento Kings,Power Forward,650000
...,...,...,...,...,...,...,...
16249,2018-2019,2019,2018,Jonathan Gibson,Boston Celtics,Shooting Guard and Point Guard,17092
16250,2018-2019,2019,2018,Tarik Phillip,Washington Wizards,Shooting Guard and Point Guard,9474
16251,2018-2019,2019,2018,Duncan Robinson,Miami Heat,Shooting Guard,9474
16252,2018-2019,2019,2018,Theo Pinson,Brooklyn Nets,Shooting Guard,4737


In [7]:
# Modern salaries data
modern_salaries_df = salaries_df.loc[salaries_df['season_start'] >= 2000]
modern_salaries_df

,season,season_end,season_start,name,team,position,salary
17,2000-01,2001,2000,Mahmoud Abdul-Rauf,Vancouver Grizzlies,Point Guard,798500
21,2000-01,2001,2000,Tariq Abdul-Wahad,Denver Nuggets,Shooting Guard,4500000
22,2001-02,2002,2001,Tariq Abdul-Wahad,Denver Nuggets,Shooting Guard,5062500
23,2002-03,2003,2002,Tariq Abdul-Wahad,Dallas Mavericks,Shooting Guard,5625000
24,2003-04,2004,2003,Tariq Abdul-Wahad,Dallas Mavericks,Shooting Guard,6187500
...,...,...,...,...,...,...,...
16249,2018-2019,2019,2018,Jonathan Gibson,Boston Celtics,Shooting Guard and Point Guard,17092
16250,2018-2019,2019,2018,Tarik Phillip,Washington Wizards,Shooting Guard and Point Guard,9474
16251,2018-2019,2019,2018,Duncan Robinson,Miami Heat,Shooting Guard,9474
16252,2018-2019,2019,2018,Theo Pinson,Brooklyn Nets,Shooting Guard,4737


In [8]:
# Modern Raptor Data 
rap_mod = os.path.join(pwd, 'Resources', 'modern_RAPTOR_by_player.csv')
mod_rap_df = pd.read_csv(rap_mod)

In [9]:
# Historical Raptor Data
rap_hist = os.path.join(pwd, 'Resources', 'historical_RAPTOR_by_player.csv')
hist_rap_df = pd.read_csv(rap_hist)

In [10]:
# check how recent is modern raptor data
print(mod_rap_df['season'].max())
print(mod_rap_df['season'].min())

2020
2014


In [11]:
# check dates for historical raptor data
print(hist_rap_df['season'].max())
print(hist_rap_df['season'].min())

2020
1977


In [12]:
hist_rap_df.head(15)

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
0,Alaa Abdelnaby,abdelal01,1991,640,303,-3.938450,-0.510076,-4.448526,-0.265191,-0.226163,-0.039028,-3.905732,-1.696796,-5.602528,0.199241
1,Alaa Abdelnaby,abdelal01,1992,1998,959,-2.553849,-0.197943,-2.751792,-0.006893,0.070322,-0.077215,-2.499576,-0.721876,-3.221452,0.485436
2,Alaa Abdelnaby,abdelal01,1993,2754,1379,-2.373736,-2.069808,-4.443544,-1.227189,-0.845272,-0.381918,-2.328144,-2.002263,-4.330407,0.488266
3,Alaa Abdelnaby,abdelal01,1994,320,159,-6.140056,-2.748312,-8.888368,-0.500600,-0.500600,0.000000,-5.820936,-3.526560,-9.347496,0.511152
4,Alaa Abdelnaby,abdelal01,1995,984,506,-3.846543,-1.268012,-5.114556,-0.615802,-0.615802,0.000000,-3.715429,-1.027024,-4.742453,1.545922
5,Kareem Abdul-Jabbar,abdulka01,1977,7674,3483,4.544044,3.103855,7.647899,18.488255,15.456216,3.032039,4.762807,2.624501,7.387308,-0.502962
6,Kareem Abdul-Jabbar,abdulka01,1978,5343,2399,3.447574,3.423490,6.871064,11.688727,11.478972,0.209755,3.944529,2.820569,6.765097,-0.566163
7,Kareem Abdul-Jabbar,abdulka01,1979,7826,3524,2.950435,2.534175,5.484610,14.739138,13.203270,1.535868,3.238178,2.029599,5.267777,-0.799426
8,Kareem Abdul-Jabbar,abdulka01,1980,8226,3761,3.121840,2.430631,5.552471,15.920002,12.849811,3.070190,3.409151,1.682673,5.091824,-0.726437
9,Kareem Abdul-Jabbar,abdulka01,1981,6716,3110,2.374990,1.802589,4.177579,10.928773,10.599289,0.329484,2.545213,1.212412,3.757625,-0.609847


In [13]:
mod_rap_df.head(15)

,player_name,player_id,season,poss,mp,raptor_box_offense,raptor_box_defense,raptor_box_total,raptor_onoff_offense,raptor_onoff_defense,...,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
0,Alex Abrines,abrinal01,2017,2387,1135,0.745505,-0.372938,0.372567,-0.418553,-3.857011,...,0.543421,-1.144832,-0.601411,1.249008,1.447708,-0.198700,0.077102,-1.038677,-0.961575,0.326413
1,Alex Abrines,abrinal01,2018,2546,1244,0.317549,-1.725325,-1.407776,-1.291727,-0.049694,...,-0.020826,-1.502642,-1.523468,0.777304,0.465912,0.311392,-0.174621,-1.112625,-1.287247,-0.456141
2,Alex Abrines,abrinal01,2019,1279,588,-3.215683,1.078399,-2.137285,-6.158856,4.901168,...,-4.040157,1.885618,-2.154538,0.178167,0.178167,0.000000,-4.577678,1.543282,-3.034396,-0.268013
3,Quincy Acy,acyqu01,2014,1716,847,-1.716079,0.133115,-1.582964,-0.324811,-1.661495,...,-1.565257,-0.216477,-1.781734,0.415918,0.415918,0.000000,-1.464417,-0.223754,-1.688171,-0.554898
4,Quincy Acy,acyqu01,2015,2517,1287,-2.014956,-1.268440,-3.283396,-3.855207,2.803842,...,-2.456065,-0.420948,-2.877013,-0.083313,-0.083313,0.000000,-2.797224,-1.648024,-4.445248,-0.100454
5,Quincy Acy,acyqu01,2016,1852,876,-0.008328,0.340783,0.332456,-2.798490,0.130301,...,-0.587540,0.353933,-0.233607,1.123479,1.123479,0.000000,-1.406108,0.021970,-1.384138,-0.102988
6,Quincy Acy,acyqu01,2017,1169,558,-0.129002,0.444339,0.315337,-1.345267,0.806701,...,-0.343630,0.576150,0.232520,0.835632,0.835632,0.000000,-0.922087,-0.059177,-0.981264,0.125008
7,Quincy Acy,acyqu01,2018,2856,1359,-2.620032,-0.806269,-3.426301,0.055170,-0.019195,...,-2.208182,-0.658814,-2.866997,-0.081232,-0.081232,0.000000,-1.826813,-0.890246,-2.717059,0.136139
8,Quincy Acy,acyqu01,2019,278,123,-5.741317,3.253365,-2.487951,-3.308027,10.098732,...,-5.581744,4.969011,-0.612733,0.135231,0.135231,0.000000,-5.877811,2.495748,-3.382063,0.841981
9,Jaylen Adams,adamsja01,2019,952,428,-2.679920,-4.592002,-7.271921,-0.535827,0.617508,...,-2.377644,-3.734729,-6.112373,-0.733513,-0.733513,0.000000,-2.023145,-4.304853,-6.327998,-0.099018


In [14]:
hist_rap_df.loc[hist_rap_df['player_name'] == 'Alex Abrines']

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
52,Alex Abrines,abrinal01,2017,2387,1135,0.543421,-1.144832,-0.601411,1.249008,1.447708,-0.198700,0.077102,-1.038677,-0.961575,0.326413
53,Alex Abrines,abrinal01,2018,2546,1244,-0.020826,-1.502642,-1.523468,0.777304,0.465912,0.311392,-0.174621,-1.112625,-1.287247,-0.456141
54,Alex Abrines,abrinal01,2019,1279,588,-4.040157,1.885618,-2.154538,0.178167,0.178167,0.000000,-4.577678,1.543282,-3.034396,-0.268013


In [15]:
two_thousands_rap_df = hist_rap_df.loc[hist_rap_df['season'] >= 2000]
two_thousands_rap_df

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
26,Mahmoud Abdul-Rauf,abdulma02,2001,939,486,0.678521,-2.640498,-1.961977,0.195461,0.195461,0.000000,-0.031734,-3.090444,-3.122178,0.030938
29,Tariq Abdul-Wahad,abdulta01,2000,3215,1578,-1.188052,-0.457694,-1.645746,0.876071,0.876071,0.000000,-1.310627,-0.196093,-1.506720,0.018992
30,Tariq Abdul-Wahad,abdulta01,2001,820,420,-5.303661,-0.963653,-6.267315,-0.758930,-0.758930,0.000000,-5.359760,-1.405260,-6.765020,0.640190
31,Tariq Abdul-Wahad,abdulta01,2002,841,441,-2.524324,-0.478045,-3.002369,-0.056546,-0.056546,0.000000,-2.679625,-0.135060,-2.814685,0.412453
32,Tariq Abdul-Wahad,abdulta01,2003,553,283,-0.212466,-0.957466,-1.169932,0.228929,0.199733,0.029195,-0.861448,-1.391198,-2.252645,-0.325525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18007,Ante Zizic,zizican01,2020,468,221,-2.525090,-0.593045,-3.118135,-0.041255,-0.041255,0.000000,-2.915056,-1.772787,-4.687843,-0.625340
18009,Ivica Zubac,zubaciv01,2017,1305,609,-3.358178,-3.040589,-6.398767,-1.141160,-1.141160,0.000000,-3.533604,-2.932103,-6.465707,0.644128
18010,Ivica Zubac,zubaciv01,2018,871,410,-2.903709,2.688832,-0.214877,0.531968,0.531968,0.000000,-2.616331,0.752875,-1.863456,0.311576
18011,Ivica Zubac,zubaciv01,2019,2345,1079,-2.362444,1.813768,-0.548676,1.217134,1.114995,0.102138,-2.810283,1.157824,-1.652459,0.077074


In [16]:
two_thousands_rap_df['season'].min()

2000

In [20]:
two_thousands_rap_df.columns.tolist()

['player_name',
 'player_id',
 'season',
 'poss',
 'mp',
 'raptor_offense',
 'raptor_defense',
 'raptor_total',
 'war_total',
 'war_reg_season',
 'war_playoffs',
 'predator_offense',
 'predator_defense',
 'predator_total',
 'pace_impact']

In [22]:
two_thousands_rap_df= two_thousands_rap_df.drop(columns=['poss', 'war_reg_season', 'war_playoffs','predator_offense', 
                                   'predator_defense','pace_impact', 'predator_total'])

In [23]:
two_thousands_rap_df

,player_name,player_id,season,mp,raptor_offense,raptor_defense,raptor_total,war_total
26,Mahmoud Abdul-Rauf,abdulma02,2001,486,0.678521,-2.640498,-1.961977,0.195461
29,Tariq Abdul-Wahad,abdulta01,2000,1578,-1.188052,-0.457694,-1.645746,0.876071
30,Tariq Abdul-Wahad,abdulta01,2001,420,-5.303661,-0.963653,-6.267315,-0.758930
31,Tariq Abdul-Wahad,abdulta01,2002,441,-2.524324,-0.478045,-3.002369,-0.056546
32,Tariq Abdul-Wahad,abdulta01,2003,283,-0.212466,-0.957466,-1.169932,0.228929
...,...,...,...,...,...,...,...,...
18007,Ante Zizic,zizican01,2020,221,-2.525090,-0.593045,-3.118135,-0.041255
18009,Ivica Zubac,zubaciv01,2017,609,-3.358178,-3.040589,-6.398767,-1.141160
18010,Ivica Zubac,zubaciv01,2018,410,-2.903709,2.688832,-0.214877,0.531968
18011,Ivica Zubac,zubaciv01,2019,1079,-2.362444,1.813768,-0.548676,1.217134


In [25]:
path = os.path.join(pwd, 'Resources', 'playermainstats.csv')
main_stats = pd.read_csv(path)
main_stats

,id,full_name,first_name,last_name,GROUP_VALUE,TEAM_ABBREVIATION,MIN,FG_PCT,FG3_PCT,FT_PCT,REB,AST,TOV,STL,BLK,PTS
0,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,2000-01,VAN,485.171667,0.488,0.286,0.759,25,76,26,9,1,266
1,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,1997-98,SAC,530.588333,0.377,0.161,1.000,37,58,19,16,1,227
2,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,1996-97,SAC,2119.933333,0.445,0.382,0.846,122,189,119,56,6,1031
3,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,2002-03,DAL,203.583333,0.466,0.000,0.500,40,21,7,6,3,57
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,2001-02,TOT,442.403333,0.374,0.500,0.727,84,24,27,20,10,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14443,1627826,Ivica Zubac,Ivica,Zubac,2018-19,TOT,1039.200000,0.559,0.000,0.802,362,63,70,14,51,525
14444,1627826,Ivica Zubac,Ivica,Zubac,2018-19,LAC,523.498333,0.538,0.000,0.733,200,38,37,10,24,244
14445,1627826,Ivica Zubac,Ivica,Zubac,2018-19,LAL,515.701667,0.580,0.000,0.864,162,25,33,4,27,281
14446,1627826,Ivica Zubac,Ivica,Zubac,2017-18,LAL,409.821667,0.500,0.000,0.765,123,25,26,8,15,161


In [26]:
recent_stats = main_stats[main_stats['GROUP_VALUE'].str.contains(r'2(?!$)')]
recent_stats

,id,full_name,first_name,last_name,GROUP_VALUE,TEAM_ABBREVIATION,MIN,FG_PCT,FG3_PCT,FT_PCT,REB,AST,TOV,STL,BLK,PTS
0,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,2000-01,VAN,485.171667,0.488,0.286,0.759,25,76,26,9,1,266
3,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,2002-03,DAL,203.583333,0.466,0.000,0.500,40,21,7,6,3,57
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,2001-02,TOT,442.403333,0.374,0.500,0.727,84,24,27,20,10,135
5,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,2001-02,DAL,23.805000,0.000,0.000,0.000,6,2,3,2,1,0
6,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,2001-02,DEN,418.598333,0.379,0.500,0.750,78,22,24,18,9,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14443,1627826,Ivica Zubac,Ivica,Zubac,2018-19,TOT,1039.200000,0.559,0.000,0.802,362,63,70,14,51,525
14444,1627826,Ivica Zubac,Ivica,Zubac,2018-19,LAC,523.498333,0.538,0.000,0.733,200,38,37,10,24,244
14445,1627826,Ivica Zubac,Ivica,Zubac,2018-19,LAL,515.701667,0.580,0.000,0.864,162,25,33,4,27,281
14446,1627826,Ivica Zubac,Ivica,Zubac,2017-18,LAL,409.821667,0.500,0.000,0.765,123,25,26,8,15,161


In [27]:
salary_path = os.path.join(pwd, 'Recources', 'salaries_1985to2022.csv')
salaries_df = pd.read_csv(salaries_path)
salaries_df

,season,season_end,season_start,name,team,position,salary
0,1990-91,1991,1990,Alaa Abdelnaby,Portland Trail Blazers,Power Forward,395000
1,1991-92,1992,1991,Alaa Abdelnaby,Portland Trail Blazers,Power Forward,494000
2,1992-93,1993,1992,Alaa Abdelnaby,Boston Celtics,Power Forward,500000
3,1993-94,1994,1993,Alaa Abdelnaby,Boston Celtics,Power Forward,805000
4,1994-95,1995,1994,Alaa Abdelnaby,Sacramento Kings,Power Forward,650000
...,...,...,...,...,...,...,...
16249,2018-2019,2019,2018,Jonathan Gibson,Boston Celtics,Shooting Guard and Point Guard,17092
16250,2018-2019,2019,2018,Tarik Phillip,Washington Wizards,Shooting Guard and Point Guard,9474
16251,2018-2019,2019,2018,Duncan Robinson,Miami Heat,Shooting Guard,9474
16252,2018-2019,2019,2018,Theo Pinson,Brooklyn Nets,Shooting Guard,4737


In [28]:
recent_salaries_df = salaries_df.loc[salaries_df['season_start'] >= 2000]
recent_salaries_df

,season,season_end,season_start,name,team,position,salary
17,2000-01,2001,2000,Mahmoud Abdul-Rauf,Vancouver Grizzlies,Point Guard,798500
21,2000-01,2001,2000,Tariq Abdul-Wahad,Denver Nuggets,Shooting Guard,4500000
22,2001-02,2002,2001,Tariq Abdul-Wahad,Denver Nuggets,Shooting Guard,5062500
23,2002-03,2003,2002,Tariq Abdul-Wahad,Dallas Mavericks,Shooting Guard,5625000
24,2003-04,2004,2003,Tariq Abdul-Wahad,Dallas Mavericks,Shooting Guard,6187500
...,...,...,...,...,...,...,...
16249,2018-2019,2019,2018,Jonathan Gibson,Boston Celtics,Shooting Guard and Point Guard,17092
16250,2018-2019,2019,2018,Tarik Phillip,Washington Wizards,Shooting Guard and Point Guard,9474
16251,2018-2019,2019,2018,Duncan Robinson,Miami Heat,Shooting Guard,9474
16252,2018-2019,2019,2018,Theo Pinson,Brooklyn Nets,Shooting Guard,4737


In [17]:
# They have an equation they give which calculates WAR from RAPTOR.
# Basically, if I'm interpreting correctly, RAPTOR is the assessment of the play, 
# WAR is cumulative value relative to a replacement player.
# "WAR" has proven a confusing name to give a stat in the NBA because all the metrics have 
# a variation attempting to estimate wins above replacement.

In [18]:
# next steps: 
# merge salaries with stats in Postgres from 2000-current
# load in merged data
# clean raptor data so only has raptor offense, defense, total, war total
# add cleaned raptor to salary stats table
# clean table so it is easy to read with only a few features
# build dashboard

In [19]:
# from sqlalchemy import create_engine
# import psycopg2 
# import io
# from config import password
# import time

# DB_address = 'nbadb.ca9dadq6ltaa.us-east-2.rds.amazonaws.com'
# engine = create_engine(f'postgresql://team:{password}@{DB_address}:5432/NBA_database')
# gamelog_players.to_sql('playerstatsbyseason', engine, if_exists='replace',index=False) #drops old table and creates new empty table

# # import the ratings data in chunks because it is too big 
# # create a variable for the number of rows imported
# rows_imported = 0
# # gets the start time from time.time()
# start_time = time.time()
# for data in pd.read_csv(f'{pwd}/playerstatsbyseason.csv', chunksize=1000000):

#     # print out the range of rows that are being imported
#     print(f'importing rows {rows_imported} to {rows_imported + len(data)}...', end='')
#     data.to_sql(name='playerstatsbyseason', con=engine, if_exists='append', index=False)

#     # increment the number of rows imported by the chunksize
#     rows_imported += len(data)
#     # print that the rows have finished importing
#     print(f'Done. {time.time()- start_time} total seconds elapsed')

NameError: name 'gamelog_players' is not defined